Réseau de neurones à une couche

Soit le jeu de données (X, y).

Le réseau de neurones a une couche est défini par: $\hat{y} = f(W X + B)$

Avec:

 - W: poids des neurones
 - B: biais
 - f: fonction d'activation non linéaire
 

Dans le cas d'une régression, 
 
On cherche à minimiser l'erreur E:

 - Mean Square Error (régression): $E = \frac{1}{2}(y - \hat{y})^2$
 - Cross-Entropy: $L_W(\hat(y), y) = -\sum y_c log(\hat{y}_c) = -log(\hat{y}_{c^*}) $
 - Divergence de Kullback-Leibler

On va minimiser par **descente de gradient**:

$W^h = W^{h-1} - \epsilon \frac {\partial E} {\partial W^{h-1}}$

On calcule les dérivées partielles (chain rule):

$\frac {\partial E} {\partial W^{h-1}} = \frac {\partial E} {\partial W^{h-1}} \frac {\partial E} {\partial W^{h-1}}$


In [ ]:
!ls iris.data || wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -o iris.data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("iris.data", header=None, names=["sepal length", "sepal width", "petal length", "petal width", "class"], dtype={"class": 'category'})

In [ ]:
# Transforme une sortie en une distribution de probabilité
def softmax(s):
    return np.exp(s)/np.sum(np.exp(s))

In [ ]:
# Sigmoid
def 𝜎(x):
    return 1/(1+np.exp((-x).tolist()))

In [ ]:
num_features = data.shape[1] - 1
categories = data["class"].cat.categories.to_numpy()
num_class = len(categories)
W = np.random.random((num_class, num_features))
# Biais
W_b = np.random.random((num_class,))

## Forward

In [ ]:
x = data.iloc[0][:4].to_numpy().astype('float64')
h = W @ x + W_b
# Fonction d'activation
s = 𝜎(h)
ŷ = softmax(s)
ŷ

In [ ]:
y = categories == data.iloc[0]["class"]
i = np.argmax(y)
error = np.sum((y - ŷ)**2)
error

## Backward / Rétropropagation de l'erreur

On cherche à minimiser l'erreur. Pouce faire, on va utiliser une descente de gradient.

L'erreur est égale à:
$
\epsilon = \frac {1} {2} ||ŷ - model(x)||^2
$

Et sa dérivée:
$
\delta_\epsilon = \frac {d \epsilon} {d W} = \frac {\frac {1} {2} ||ŷ - model(x)||^2} {d W}
$

Pour calculer ce résultat, nous allons utiliser la **dérivation de fonctions composées** *(f ◦ g ◦ h)(z)* ou "chain rule" en anglais:

$
\delta_\epsilon = \frac {\frac {1} {2} ||ŷ - y||^2} {d y} * \frac {d softmax(s)} {d s} * \frac {d \sigma(h)} {d h} * \frac {W * x + W_b} {W}
$

Avec:
 * $ \frac {\frac {1} {2} ||ŷ - y||^2} {d y} = ||ŷ - y||$
 * $ \frac {d \sigma(h)} {d h} = \sigma(h)(1−\sigma(h))$
 * $ \frac {d W * x + W_b} {d W} = x$
 
La [dérivée de softmax][1] est un peu compliquée à calculer. On peut simplifier le calcul en utilisant comme fonction d'erreur l'entropie croisée $\mathcal{L}(y, p) = - \sum_i y_i log(p_i)$. Comme $y_i$ vaut 1 seulement pour la classe considérée, on obtient pour la [dérivée de softmax avec l'entropie croisée](https://deepnotes.io/softmax-crossentropy):

$ \frac {\mathcal{L}(y, ŷ)} {d y} * \frac {d softmax(s)} {d s} = \hat{y}_i - y_i$


Référence:
 * [1](https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1)
 * [2](https://deepnotes.io/softmax-crossentropy)

In [ ]:
𝛿 = np.transpose(x.reshape((1,-1))) @ ((ŷ[i] - 1) * (𝜎(h) * (1 - 𝜎(h)))).reshape((1,-1))
𝛿

Pour effectuer une descente de gradient, nous allons soustraire le gradient calculé au paramètre du modèle selon un hyperparamètre $\alpha$ appellé le taux d'apprentissage:

In [ ]:
𝛼 = 0.05
W -= 𝛼 * 𝛿.T
W

In [ ]:
h = W @ x + W_b
# Fonction d'activation
s = 𝜎(h)
ŷ = softmax(s)
error = np.sum((y - ŷ)**2)
error